In [23]:
# Importing necessary libraries
import pandas as pd
import numpy as np


from matplotlib import pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn'


# from transformers import SentenceTransformer
import numpy as np
import json
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
from collections import OrderedDict


import pyarrow.parquet as pa

In [24]:
df = pd.read_csv('/home/mendu/Thesis/data/magnatagatune/saved_df_data/processed_df_ML.csv', index_col=[0])

In [25]:
df

,mp3_path,tags
clip_id,,
2,american_bach_soloists-j_s__bach_solo_cantatas...,"['opera', 'classical']"
6,american_bach_soloists-j_s__bach_solo_cantatas...,"['opera', 'classical']"
10,american_bach_soloists-j_s__bach_solo_cantatas...,"['opera', 'classical']"
11,american_bach_soloists-j_s__bach_solo_cantatas...,['opera']
12,american_bach_soloists-j_s__bach_solo_cantatas...,['classical']
...,...,...
58897,jacob_heringman-blame_not_my_lute-56-la_bressa...,['classical']
58898,jacob_heringman-blame_not_my_lute-56-la_bressa...,['classical']
58907,jacob_heringman-blame_not_my_lute-57-lost_is_m...,['classical']


In [26]:
table_test = pa.read_table('/home/mendu/Thesis/data/magnatagatune/captions/test-00000-of-00001-94781ef88fa7ed89.parquet') 
table_train = pa.read_table('/home/mendu/Thesis/data/magnatagatune/captions/train-00000-of-00001-28dbf9154d6d526d.parquet') 
table_val = pa.read_table('/home/mendu/Thesis/data/magnatagatune/captions/valid-00000-of-00001-bf9893b31ca2d5e5.parquet') 

In [27]:
df_test = table_test.to_pandas().set_index('track_id')
df_train = table_train.to_pandas().set_index('track_id')
df_val = table_val.to_pandas().set_index('track_id') 

In [28]:
len(df_train)+len(df_test)+len(df_val)

25860

In [29]:
frames = [df_test, df_train, df_val]
df_captions = pd.concat(frames)

In [30]:
captions = df_captions[['caption_writing']]
captions.index = captions.index.astype('int64')
captions

,caption_writing
track_id,
2,Experience the majestic beauty of classical mu...
6,Experience the rich sound of classical eleganc...
10,This powerful classic opera piece showcases th...
11,This atmospheric and introspective song blends...
12,Experience a powerful and uptempo classical me...
...,...
58716,This breathtaking song features a mesmerizing ...
58717,This folk-inspired song features intricate str...
58719,This hauntingly beautiful ballad takes its tim...


In [31]:
df = pd.merge(df, captions, how = 'left', left_index=True, right_index=True)


In [32]:
df

,mp3_path,tags,caption_writing
clip_id,,,
2,american_bach_soloists-j_s__bach_solo_cantatas...,"['opera', 'classical']",Experience the majestic beauty of classical mu...
6,american_bach_soloists-j_s__bach_solo_cantatas...,"['opera', 'classical']",Experience the rich sound of classical eleganc...
10,american_bach_soloists-j_s__bach_solo_cantatas...,"['opera', 'classical']",This powerful classic opera piece showcases th...
11,american_bach_soloists-j_s__bach_solo_cantatas...,['opera'],This atmospheric and introspective song blends...
12,american_bach_soloists-j_s__bach_solo_cantatas...,['classical'],Experience a powerful and uptempo classical me...
...,...,...,...
58897,jacob_heringman-blame_not_my_lute-56-la_bressa...,['classical'],This beautiful classical piece features a haun...
58898,jacob_heringman-blame_not_my_lute-56-la_bressa...,['classical'],This classical piece features beautiful melodi...
58907,jacob_heringman-blame_not_my_lute-57-lost_is_m...,['classical'],This beautiful classical piece features a gent...


In [33]:
# df.to_csv('/home/mendu/Thesis/data/magnatagatune/saved_df_data/df_w_captions.csv')

## Getting the embedding for the caption

In [34]:
# Define the Encoder class focusing only on the encoding part
class Encoder(nn.Module):
    def __init__(self, input_size, encoding_size):
        super(Encoder, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(True),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.BatchNorm1d(128),
            nn.Linear(128, encoding_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

In [36]:
# Define input and encoding sizes
input_size = 768
encoding_size = 64
# encoding_size = 128

In [15]:
def load_encoder_state_dict(encoder, state_dict_path):
    state_dict = torch.load(state_dict_path)
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = f'layers.{k}'
        new_state_dict[name] = v
    
    encoder.load_state_dict(new_state_dict)
    return encoder

def get_device():
    return torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

device = get_device()

In [37]:
# Instantiate the encoder
encoder = Encoder(input_size=input_size, encoding_size=encoding_size).to(device)
encoder.eval()  # Set the encoder to evaluation mode

# Load the pre-trained weights for the encoder
encoder = load_encoder_state_dict(encoder, '/home/mendu/Thesis/data/musiccaps/auto_encoder/encoder_state_dict64.pth')
# encoder = load_encoder_state_dict(encoder, '/home/mendu/Thesis/data/musiccaps/auto_encoder/encoder_state_dict128.pth')

# Load the SentenceTransformer model and move to the correct device
roberta_model = SentenceTransformer('/home/mendu/Thesis/data/musiccaps/new_embedding_model2').to(device)

# Taking a sample caption from df
caption = df.caption_writing[2]

In [38]:
# Function to encode caption
def encode_caption(encoder, sentence_model, text):
    with torch.no_grad():
        encoded_caption = sentence_model.encode(text, convert_to_tensor=True)
        encoded_caption = encoded_caption.to(device)
        return encoder(encoded_caption.unsqueeze(0))

In [39]:
# Encoded caption 
fully_encoded_caption = encode_caption(encoder, roberta_model, caption)

In [40]:
fully_encoded_caption

tensor([[1.8345e-03, 1.0000e+00, 1.0000e+00, 2.6551e-06, 1.5006e-12, 9.9900e-01,
         7.0840e-01, 5.1398e-09, 1.0000e+00, 1.0000e+00, 1.3518e-05, 1.0000e+00,
         3.7229e-09, 8.4054e-01, 1.8887e-06, 9.9106e-01, 6.3480e-01, 9.9999e-01,
         1.0000e+00, 1.6942e-12, 1.7556e-06, 1.0000e+00, 9.9958e-01, 2.1419e-01,
         6.4560e-07, 5.5366e-07, 9.5301e-01, 9.9948e-01, 2.0237e-01, 1.8981e-01,
         9.9993e-01, 6.3691e-09, 2.8534e-08, 3.7270e-09, 1.7842e-05, 1.0000e+00,
         1.0000e+00, 9.9997e-01, 1.0000e+00, 8.3582e-01, 1.7160e-10, 2.7980e-03,
         1.6188e-05, 1.0507e-09, 1.0000e+00, 1.0000e+00, 8.2446e-06, 1.0000e+00,
         9.7433e-01, 2.4049e-08, 1.7615e-03, 1.3273e-13, 9.3549e-04, 1.1792e-17,
         2.0446e-11, 9.9998e-01, 5.9708e-02, 1.7616e-17, 5.6193e-04, 9.3821e-01,
         8.1709e-01, 9.9952e-01, 1.4021e-05, 1.0000e+00]], device='cuda:0')

In [41]:
df.head()

,mp3_path,tags,caption_writing
clip_id,,,
2,american_bach_soloists-j_s__bach_solo_cantatas...,"['opera', 'classical']",Experience the majestic beauty of classical mu...
6,american_bach_soloists-j_s__bach_solo_cantatas...,"['opera', 'classical']",Experience the rich sound of classical eleganc...
10,american_bach_soloists-j_s__bach_solo_cantatas...,"['opera', 'classical']",This powerful classic opera piece showcases th...
11,american_bach_soloists-j_s__bach_solo_cantatas...,['opera'],This atmospheric and introspective song blends...
12,american_bach_soloists-j_s__bach_solo_cantatas...,['classical'],Experience a powerful and uptempo classical me...


In [42]:
# Apply the encode_caption function to each row's caption_writing
def get_encoded_caption(row):
    caption = row['caption_writing']
    return encode_caption(encoder, roberta_model, caption).cpu()

df['caption_writing'] = df['caption_writing'].astype(str)

# Apply the encoding function to the DataFrame
df['caption_embedding'] = df.apply(get_encoded_caption, axis=1)

# Flatten the nested list
df['caption_embedding'] = df['caption_embedding'].apply(lambda x: np.array(x)[0])

# Now df has an additional column with the caption embeddings as tensors
df

,mp3_path,tags,caption_writing,caption_embedding
clip_id,,,,
2,american_bach_soloists-j_s__bach_solo_cantatas...,"['opera', 'classical']",Experience the majestic beauty of classical mu...,"[0.0018344965, 1.0, 1.0, 2.655105e-06, 1.50058..."
6,american_bach_soloists-j_s__bach_solo_cantatas...,"['opera', 'classical']",Experience the rich sound of classical eleganc...,"[0.00030539755, 1.0, 1.0, 7.7272097e-07, 1.570..."
10,american_bach_soloists-j_s__bach_solo_cantatas...,"['opera', 'classical']",This powerful classic opera piece showcases th...,"[0.0017760618, 1.0, 1.0, 5.9899867e-06, 4.3203..."
11,american_bach_soloists-j_s__bach_solo_cantatas...,['opera'],This atmospheric and introspective song blends...,"[0.00037677313, 1.0, 1.0, 3.5533972e-07, 2.639..."
12,american_bach_soloists-j_s__bach_solo_cantatas...,['classical'],Experience a powerful and uptempo classical me...,"[0.013908892, 1.0, 0.9999993, 0.000117422234, ..."
...,...,...,...,...
58897,jacob_heringman-blame_not_my_lute-56-la_bressa...,['classical'],This beautiful classical piece features a haun...,"[0.00088836084, 1.0, 1.0, 1.0079755e-06, 5.713..."
58898,jacob_heringman-blame_not_my_lute-56-la_bressa...,['classical'],This classical piece features beautiful melodi...,"[0.0050765076, 1.0, 1.0, 9.212903e-06, 2.80899..."
58907,jacob_heringman-blame_not_my_lute-57-lost_is_m...,['classical'],This beautiful classical piece features a gent...,"[0.0013480721, 1.0, 1.0, 1.892443e-06, 1.53413..."


In [50]:
problematic_files = ['jacob_heringman-josquin_des_prez_lute_settings-19-gintzler__pater_noster-204-233.mp3',
 'american_baroque-dances_and_suites_of_rameau_and_couperin-25-le_petit_rien_xiveme_ordre_couperin-88-117.mp3']

In [52]:
df = df[~df['mp3_path'].isin(problematic_files)]

In [53]:
# Save to CSV
df.to_csv('/home/mendu/Thesis/data/magnatagatune/saved_df_data/df_w_embeddings_ML.csv', index=False)

# Save as pickle file
df.to_pickle("/home/mendu/Thesis/data/magnatagatune/saved_df_data/df_w_embeddings_ML.pkl")